In [ ]:
# default_exp cleaner

In [11]:
#export
import numpy as np
import pandas as pd
from pandas.api.types import  is_numeric_dtype, is_integer_dtype
import matplotlib.pyplot as plt
import re
from fastai.tabular.all import *

In [5]:
%reload_ext autoreload
%autoreload 2

In [8]:
filename = "Buldozers.csv"           # dataframe containing all the data
y_col_name = 'SalePrice'         # name of the column we are going to predict
split_pct = 0.2             # size of verification dataset
datefield = 'saledate'     # name of the field containing the date-part

In [9]:
#export
def fix_dataframe(df, y_col_name, datefield=None, ):
    df_shrink_dtypes(df)
    if datefield is not None:
        df.sort_values(datefield, inplace=True)
    df[y_col_name] = np.log(df[y_col_name])
    return df   

In [12]:
df = pd.read_csv(filename, low_memory=False)
df = fix_dataframe(df, datefield=datefield, y_col_name=y_col_name)

In [16]:
#export
def split_by_date(df, pct):
    validx_start = int(len(df)*(1-pct))
    train_split = list(range(0, validx_start))
    val_split = list(range(validx_start, len(df)))
    return (train_split, val_split)

In [14]:
splits = split_by_date(df, pct=0.2)
len(splits[0]), len(splits[1])

(80224, 20057)

In [3]:
#export
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [4]:
df = pd.read_csv("Buldozers.csv", low_memory=False)
display_all(df)

,Unnamed: 0,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,Track Excavators,NaN,EROPS w AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,EROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100276,100276,1418366,56000,1111855,3357,132,1.0,1994,NaN,NaN,12/7/2001 0:00,12G,12,G,NaN,NaN,NaN,Motorgrader - 130.0 to 145.0 Horsepower,Georgia,MG,Motor Graders,No,EROPS,NaN,NaN,NaN,NaN,None or Unspecified,NaN,None or Unspecified,14',None or Unspecified,No,Base + 1 Function,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,"15.5""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100277,100277,1418367,130000,20922,3357,132,6.0,1994,NaN,NaN,5/10/1996 0:00,12G,12,G,NaN,NaN,NaN,Motorgrader - 130.0 to 145.0 Horsepower,Tennessee,MG,Motor Graders,No,EROPS w AC,NaN,NaN,NaN,NaN,None or Unspecified,NaN,None or Unspecified,None or Unspecified,None or Unspecified,No,Base + 1 Function,None or Unspecified,None or Unspecified,Yes,Sideshift & Tip,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100278,100278,1418368,70000,1551595,3357,132,1.0,1994,NaN,NaN,9/27/2006 0:00,12G,12,G,NaN,NaN,NaN,Motorgrader - 130.0 to 145.0 Horsepower,Texas,MG,Motor Graders,No,EROPS w AC,NaN,NaN,NaN,NaN,None or Unspecified,NaN,None or Unspecified,14',None or Unspecified,No,Base + 1 Function,Yes,Yes,None or Unspecified,None or Unspecified,"14""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

# Handle Dates

In [5]:
#export
def add_datepart(df, fldnames, drop=True, time=False, errors="raise"):	
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string or list of strings that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.
    Examples:
    ---------
    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df
        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13
    >>> add_datepart(df, 'A')
    >>> df
        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    >>>df2 = pd.DataFrame({'start_date' : pd.to_datetime(['3/11/2000','3/13/2000','3/15/2000']),
                            'end_date':pd.to_datetime(['3/17/2000','3/18/2000','4/1/2000'],infer_datetime_format=True)})
    >>>df2
        start_date	end_date    
    0	2000-03-11	2000-03-17
    1	2000-03-13	2000-03-18
    2	2000-03-15	2000-04-01
    >>>add_datepart(df2,['start_date','end_date'])
    >>>df2
    	start_Year	start_Month	start_Week	start_Day	start_Dayofweek	start_Dayofyear	start_Is_month_end	start_Is_month_start	start_Is_quarter_end	start_Is_quarter_start	start_Is_year_end	start_Is_year_start	start_Elapsed	end_Year	end_Month	end_Week	end_Day	end_Dayofweek	end_Dayofyear	end_Is_month_end	end_Is_month_start	end_Is_quarter_end	end_Is_quarter_start	end_Is_year_end	end_Is_year_start	end_Elapsed
    0	2000	    3	        10	        11	        5	            71	            False	            False	                False	                False	                False	            False	            952732800	    2000	    3	        11	        17	    4	            77	            False	            False	            False	            False	                False	        False	            953251200
    1	2000	    3	        11	        13	        0	            73	            False	            False	                False	                False               	False           	False           	952905600     	2000       	3	        11      	18  	5           	78          	False	            False           	False           	False               	False          	False           	953337600
    2	2000	    3	        11	        15	        2           	75          	False           	False               	False               	False               	False               False           	953078400      	2000    	4          	13      	1   	5           	92          	False           	True            	False           	True                	False          	False           	954547200
    """
    if isinstance(fldnames,str): 
        fldnames = [fldnames]
        
    for fldname in fldnames:
        fld = df[fldname]
        fld_dtype = fld.dtype
        if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
            fld_dtype = np.datetime64

        if not np.issubdtype(fld_dtype, np.datetime64):
            df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
        targ_pre = re.sub('[Dd]ate$', '', fldname)
        attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
                'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
        if time: 
            attr = attr + ['Hour', 'Minute', 'Second']
        for n in attr:
            if n=='Week':
                df[targ_pre + n] = getattr(fld.dt.isocalendar(), n.lower())
            else:
                df[targ_pre + n] = getattr(fld.dt, n.lower())
        df[targ_pre + 'Elapsed'] = fld.view(np.int64) // 10 ** 9
        if drop: 
            df.drop(fldname, axis=1, inplace=True)

def is_date(x): return np.issubdtype(x.dtype, np.datetime64)

In [6]:
df_temp = df[['saledate','Ripper']].copy()
add_datepart(df_temp, 'saledate')
assert len(df_temp.columns)==14
assert 'saledate' not in df_temp.columns

In [7]:
df_temp = df[['saledate','Ripper']].copy()
add_datepart(df_temp, 'saledate', time=True)
assert len(df_temp.columns)==17
assert 'saledate' not in df_temp.columns

# Fill Missing

In [87]:
#export
def fill_missing_numeric(df:pd.DataFrame, col:str, na_dict={None:None}):
    """
    Fills missing missing values with the median or value specified in na_dict if it's used
    
    Parameters:
    df: dataframe
    col: name of column to fill missing values
    na_dict: (optional) dict with name of the column as key specific value that is used to fill missing values
    
    returning:
    alters the dataframe, adding a 'col_na' column indicating which parameters have been filled
    return a dictionary containing the value that was used to fill missing.
    
    """
    if is_numeric_dtype(df[col]):
        if df[col].isnull().sum():
            df[col + '_na'] = df[col].isnull()
            filler = na_dict[col] if col in na_dict else df[col].median()
            filler = int(filler) if is_integer_dtype(df[col]) else filler
            df[col] = df[col].fillna(filler)
            na_dict[col] = filler
    return na_dict

def fill_missing(df, columns=None, na_dict={None:None}):
    columns = df.columns if columns is None else columns
    for col in columns:
        na_dict = fill_missing_numeric(df, col, na_dict)
    return na_dict

In [79]:
df_test = pd.DataFrame({'A':[1,2,np.nan]},dtype="Int64")
d = fill_missing_numeric(df_test, col='A')
df_test

1.5 1


,A,A_na
0,1,False
1,2,False
2,1,True


In [86]:
df_test = pd.DataFrame({'A':[1,2,np.nan]},dtype="Int64")
fill_missing(df_test)

1.5 1


{None: None, 'A': 1}

In [77]:
assert d['A'] == int(1.5)
assert len(df_test.columns)==2
assert 'A_na' in df_test.columns
assert is_numeric(df_test['A']) == True

In [80]:
#export
def df_shrink_dtypes(df, skip=[], obj2cat=True, int2uint=False):
    "Return any possible smaller data types for DataFrame columns. Allows `object`->`category`, `int`->`uint`, and exclusion."

    # 1: Build column filter and typemap
    excl_types, skip = {'category','datetime64[ns]','bool'}, set(skip)

    typemap = {'int'   : [(np.dtype(x), np.iinfo(x).min, np.iinfo(x).max) for x in (np.int8, np.int16, np.int32, np.int64)],
               'uint'  : [(np.dtype(x), np.iinfo(x).min, np.iinfo(x).max) for x in (np.uint8, np.uint16, np.uint32, np.uint64)],
               'float' : [(np.dtype(x), np.finfo(x).min, np.finfo(x).max) for x in (np.float32, np.float64, np.float128)],}
    if obj2cat: typemap['object'] = 'category'  # User wants to categorify dtype('Object'), which may not always save space
    else:       excl_types.add('object')
    
    new_dtypes = {}
    exclude = lambda dt: dt[1].name not in excl_types and dt[0] not in skip

    for c, old_t in filter(exclude, df.dtypes.items()):
        t = next((v for k,v in typemap.items() if old_t.name.startswith(k)), None)
        
        if isinstance(t, list): # Find the smallest type that fits
            if int2uint and t==typemap['int'] and df[c].min() >= 0: t=typemap['uint']
            new_t = next((r[0] for r in t if r[1]<=df[c].min() and r[2]>=df[c].max()), None)
            if new_t and new_t == old_t: new_t = None
        else: new_t = t if isinstance(t, str) else None
            
        if new_t: new_dtypes[c] = new_t
    return new_dtypes

# Categorical data

In [92]:
#export
def cont_cat_split(df, max_card=20, dep_var=None):
    "Helper function that returns column names of cont and cat variables from given `df`."
    cont_names, cat_names = [], []
    for label in df:
        if label in L(dep_var): continue
        if ((pd.api.types.is_integer_dtype(df[label].dtype) and
            df[label].unique().shape[0] > max_card) or
            pd.api.types.is_float_dtype(df[label].dtype)):
            cont_names.append(label)
        else: cat_names.append(label)
    return cont_names, cat_names

In [ ]:
def categorify(df, cat_names):
    cat_mapping = dict()
    for cat in cat_names:
        pd.Categorical(df[cat], ordered=True)
        cat_mapping[cat] = '#na#' + df[cat].cat.categories

In [89]:
import pandas.api.types as types

In [90]:
dir(types)

['CategoricalDtype',
 'DatetimeTZDtype',
 'IntervalDtype',
 'PeriodDtype',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'infer_dtype',
 'is_array_like',
 'is_bool',
 'is_bool_dtype',
 'is_categorical',
 'is_categorical_dtype',
 'is_complex',
 'is_complex_dtype',
 'is_datetime64_any_dtype',
 'is_datetime64_dtype',
 'is_datetime64_ns_dtype',
 'is_datetime64tz_dtype',
 'is_dict_like',
 'is_dtype_equal',
 'is_extension_array_dtype',
 'is_extension_type',
 'is_file_like',
 'is_float',
 'is_float_dtype',
 'is_hashable',
 'is_int64_dtype',
 'is_integer',
 'is_integer_dtype',
 'is_interval',
 'is_interval_dtype',
 'is_iterator',
 'is_list_like',
 'is_named_tuple',
 'is_number',
 'is_numeric_dtype',
 'is_object_dtype',
 'is_period_dtype',
 'is_re',
 'is_re_compilable',
 'is_scalar',
 'is_signed_integer_dtype',
 'is_sparse',
 'is_string_dtype',
 'is_timedelta64_dtype',
 'is_timedelta64_ns_dtype',
 'is_

In [ ]:
def categories_to_codes(df):
    d = dict()
    for col in df.columns:
         if is_categorical_dtype(df[col]):
                df[col] = df[col].cat.codes
    

In [ ]:
#Fill missing categorical?
#typer av kategorier: int, få värden, borde<- finns väl inte efter som int blir float när NAN kommer in i bilden

In [ ]:
# kolla om unika värden är under xx
# Om string < 6, one-hot encoda dessa
# om int el float < 20, sortera och skapa kategorier
# om float, kolla om man behöver ta log eller exp på parametrarna?

In [17]:
from nbdev.export import *
notebook2script()

Converted 00_regressor.ipynb.
Converted 01_cleandata.ipynb.
Converted 03_baysian_reg.ipynb.
Converted Template_rf_forest.ipynb.
Converted index.ipynb.
